In [ ]:
from flask import Flask, request, jsonify
from flask_cors import CORS
import pandas as pd
import numpy as np
from binance.client import Client
from sklearn.preprocessing import MinMaxScaler
from xgboost import XGBRegressor
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Bidirectional
from sklearn.metrics import mean_absolute_error, mean_squared_error
import math

app = Flask(__name__)
CORS(app)

def fetch_binance_data(symbol='BTCUSDT', interval='1h', lookback='1000'):
    client = Client(api_key="QV042LDPdDYnjujvWj6j56AJP32cvJpgslj0po3xztO229xh6llArpXSjRWrxvYb",
                    api_secret="vaPZROabhGiDdPrz81Nw2vqdCPC5kKIlTzMiefRoM1SWDGoH9LukTLdLHvjWMWI6")
    klines = client.get_historical_klines(symbol, interval, f"{lookback} hours ago UTC")
    df = pd.DataFrame(klines, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume', 'close_time',
                                       'quote_asset_volume', 'trades', 'taker_buy_base', 'taker_buy_quote', 'ignore'])
    for col in ['high', 'low', 'close', 'volume']:
        df[col] = df[col].astype(float)
    return df

def add_technical_indicators(df):
    df['SMA_10'] = df['close'].rolling(window=10).mean()
    df['EMA_10'] = df['close'].ewm(span=10).mean()
    df['RSI'] = calculate_rsi(df['close'], 14)
    df['ATR'] = calculate_atr(df['close'], df['high'], df['low'], 14)
    df['Bollinger_Upper'], df['Bollinger_Lower'] = calculate_bollinger_bands(df['close'])
    df['MACD'] = df['close'].ewm(span=12).mean() - df['close'].ewm(span=26).mean()
    df['Signal_Line'] = df['MACD'].ewm(span=9).mean()
    df['Price_ROC'] = df['close'].pct_change(10) * 100
    df.dropna(inplace=True)
    return df

def calculate_rsi(series, window):
    delta = series.diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=window).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=window).mean()
    rs = gain / loss
    return 100 - (100 / (1 + rs))

def calculate_bollinger_bands(series, window=20, num_std_dev=2):
    rolling_mean = series.rolling(window).mean()
    rolling_std = series.rolling(window).std()
    upper_band = rolling_mean + (rolling_std * num_std_dev)
    lower_band = rolling_mean - (rolling_std * num_std_dev)
    return upper_band, lower_band

def calculate_atr(close, high, low, window):
    high_low = high - low
    high_close = (high - close.shift()).abs()
    low_close = (low - close.shift()).abs()
    tr = pd.concat([high_low, high_close, low_close], axis=1).max(axis=1)
    return tr.rolling(window=window).mean()

def preprocess_data(data, sequence_length=50):
    feature_columns = ['close', 'SMA_10', 'EMA_10', 'RSI', 'ATR', 'MACD', 'Signal_Line', 'Price_ROC']
    feature_scaler = MinMaxScaler()
    target_scaler = MinMaxScaler()
    features = data[feature_columns].fillna(0)
    target = data[['close']]
    features_scaled = feature_scaler.fit_transform(features)
    target_scaled = target_scaler.fit_transform(target)
    x, y = [], []
    for i in range(sequence_length, len(features_scaled)):
        x.append(features_scaled[i-sequence_length:i])
        y.append(target_scaled[i])
    return np.array(x), np.array(y), feature_scaler, target_scaler

def build_bidirectional_lstm_model(input_shape):
    model = Sequential([
        Bidirectional(LSTM(64, return_sequences=True), input_shape=input_shape),
        Dropout(0.2),
        Bidirectional(LSTM(32, return_sequences=False)),
        Dropout(0.2),
        Dense(16, activation='relu'),
        Dense(1)
    ])
    model.compile(optimizer='adam', loss='huber')
    return model

def calculate_accuracy_metrics(y_true, y_pred):
    actual_direction = np.diff(y_true.flatten())
    pred_direction = np.diff(y_pred.flatten())
    directional_accuracy = np.mean((actual_direction > 0) == (pred_direction > 0)) * 100
    mape = np.mean(np.abs((y_true - y_pred) / y_true)) * 100
    percentage_accuracy = 100 - mape
    return {
        "directional_accuracy": float(directional_accuracy),
        "MAPE": float(mape),
        "percentage_accuracy": float(percentage_accuracy)
    }

def evaluate_profitability_refined(y_test_actual, y_pred_actual, threshold=0.02, stop_loss=0.01, take_profit=0.03):
    actual_direction = np.diff(y_test_actual.flatten())
    predicted_direction = np.diff(y_pred_actual.flatten())
    initial_balance = 1000
    balance = initial_balance
    for actual, predicted in zip(actual_direction, predicted_direction):
        confidence = abs(predicted) / abs(actual) if actual != 0 else 0
        if confidence >= threshold:
            if predicted > 0:
                gain = actual / y_test_actual[0]
                if gain > take_profit:
                    balance *= 1 + take_profit
                elif gain < -stop_loss:
                    balance *= 1 - stop_loss
                else:
                    balance *= 1 + gain
            elif predicted < 0:
                loss = -actual / y_test_actual[0]
                if loss > take_profit:
                    balance *= 1 + take_profit
                elif loss < -stop_loss:
                    balance *= 1 - stop_loss
                else:
                    balance *= 1 + loss
    profitability = (balance - initial_balance) / initial_balance * 100
    return float(profitability)

@app.route('/predict', methods=['POST'])
def predict():
    try:
        data = request.json
        symbol = data.get('symbol', 'BTCUSDT')
        df = fetch_binance_data(symbol=symbol)
        df = add_technical_indicators(df)
        x, y, feature_scaler, target_scaler = preprocess_data(df)
        split = int(len(x) * 0.8)
        x_train, x_test = x[:split], x[split:]
        y_train, y_test = y[:split], y[split:]
        lstm_model = build_bidirectional_lstm_model((x_train.shape[1], x_train.shape[2]))
        lstm_model.fit(x_train, y_train, epochs=10, batch_size=32, validation_split=0.2, verbose=1)
        lstm_features_train = lstm_model.predict(x_train).flatten()
        lstm_features_test = lstm_model.predict(x_test).flatten()
        xgb_model = XGBRegressor(
            n_estimators=100, learning_rate=0.01, max_depth=5, min_child_weight=2,
            subsample=0.8, colsample_bytree=0.8, random_state=42
        )
        xgb_model.fit(lstm_features_train.reshape(-1, 1), y_train)
        y_pred = xgb_model.predict(lstm_features_test.reshape(-1, 1))
        y_test_actual = target_scaler.inverse_transform(y_test)
        y_pred_actual = target_scaler.inverse_transform(y_pred.reshape(-1, 1))
        accuracy_metrics = calculate_accuracy_metrics(y_test_actual, y_pred_actual)
        profitability = evaluate_profitability_refined(y_test_actual, y_pred_actual)
        return jsonify({
            "symbol": symbol,
            "actual": y_test_actual.flatten().tolist(),
            "predicted": y_pred_actual.flatten().tolist(),
            "directional_accuracy": accuracy_metrics["directional_accuracy"],
            "MAPE": accuracy_metrics["MAPE"],
            "percentage_accuracy": accuracy_metrics["percentage_accuracy"],
            "profitability": profitability
        })
    except Exception as e:
        return jsonify({"error": str(e)}), 500

if __name__ == '__main__':
    app.run(debug=False, threaded=True)


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
